In [7]:
from audioop import avg
from statistics import mean
import pandas as pd
import os

#Test
os.chdir(r'C:\Users\brand\Desktop\QuantumCashUsageTaskFiles')


path = os.getcwd()


raw = pd.read_csv('Quan_FinTx.csv')

raw.calendardate = pd.to_datetime(raw['calendardate'])


UsageByDay = raw.groupby(['name',
                         'number',
                          'machinename',
                          'WeekNum',
                          'calendardate']).agg({
                              '0.01c':   sum,
                              '0.05c':   sum,
                              '0.10c':   sum,
                              '0.25c':   sum,
                              '1.00n':   sum,
                              '5.00n':   sum,
                              '10.00n':  sum,
                              '20.00n':  sum,
                              '50.00n':  sum,
                              '100.00n': sum
                          }).reset_index()

MaxByWeek = UsageByDay.groupby(['name',
                         'number',
                          'machinename',
                          'WeekNum']).agg({

                              '0.01c':   min,
                              '0.05c':   min,
                              '0.10c':   min,
                              '0.25c':   min,
                              '1.00n':   min,
                              '5.00n':   min,
                              '10.00n':  min,
                              '20.00n':  min,
                              '50.00n':  min,
                              '100.00n': min
                          }).reset_index()


            
AvgByWeek = UsageByDay.groupby(['name',
                         'number',
                          'machinename',
                          'WeekNum']).agg({
                              '0.01c':   mean,
                              '0.05c':   mean,
                              '0.10c':   mean,
                              '0.25c':   mean,
                              '1.00n':   mean,
                              '5.00n':   mean,
                              '10.00n':  mean,
                              '20.00n':  mean,
                              '50.00n':  mean,
                              '100.00n': mean
                          }).reset_index()

Dispense = raw[raw['ChildTran'] == 'CashDispensed']

DispenseWeeklyMax = Dispense.groupby(['name',
                                      'number',
                                      'machinename',
                                      'WeekNum']).agg({
                                          '0.01c':   min,
                                          '0.05c':   min,
                                          '0.10c':   min,
                                          '0.25c':   min,
                                          '1.00n':   min,
                                          '5.00n':   min,
                                          '10.00n':  min,
                                          '20.00n':  min,
                                          '50.00n':  min,
                                          '100.00n': min
                                      }).reset_index()

DispenseWeeklyAvg = Dispense.groupby(['name',
                                      'number',
                                      'machinename',
                                      'WeekNum']).agg({
                                           '0.01c':   mean,
                                           '0.05c':   mean,
                                           '0.10c':   mean,
                                           '0.25c':   mean,
                                           '1.00n':   mean,
                                           '5.00n':   mean,
                                           '10.00n':  mean,
                                           '20.00n':  mean,
                                           '50.00n':  mean,
                                           '100.00n': mean
                                       }).reset_index()



#Format and Merge Dataframes

UsageMerge = MaxByWeek.merge(AvgByWeek, how='inner',on=['name','number','machinename','WeekNum'], suffixes=('_max','_avg'))

DispenseMerge  = DispenseWeeklyMax.merge(DispenseWeeklyAvg, how='inner',on=['name','number','machinename','WeekNum'], suffixes=('_max','_avg'))




with pd.ExcelWriter('TetheredQuantumUsageTest.xlsx') as writer:
    raw.to_excel(writer, sheet_name='RAW', index=False)
    UsageByDay.to_excel(writer, sheet_name='Usage By Day', index=False)
    UsageMerge.to_excel(writer, sheet_name='Weekly Usage Metrics', index=False, float_format='%.0f')
    DispenseMerge.to_excel(writer, sheet_name='Weekly Dispense Metrics', index=False, float_format='%.0f')
    
                    
